# Hugging Face Transformers 微调语言模型 - 问答任务

- 下载模型
- 下载数据集
- 数据预处理
- 训练超参数设置





## 下载模型

模型：[https://huggingface.co/distilbert/distilbert-base-uncased](https://huggingface.co/distilbert/distilbert-base-uncased)

将模型下载到：`/home/hengzq/workspace/modelscope/models/distilbert-base-uncased`


In [32]:
model_path = "/home/hengzq/workspace/modelscope/models/distilbert-base-uncased"

## 下载数据集

    斯坦福问答数据集（SQUAD）是一个阅读理解数据集，由众包工作者在一系列维基百科文章上提出问题组成。每个问题的答案都是相应阅读段落中的文本片段或范围，或者该问题可以能无法回答。


数据集地址：[https://huggingface.co/datasets/rajpurkar/squad_v2](https://huggingface.co/datasets/rajpurkar/squad_v2)

将数据下载到：`/home/hengzq/workspace/modelscope/datasets/squad_v2`目录


In [2]:
from datasets import load_dataset

dataset_path = "/home/hengzq/workspace/modelscope/datasets/squad_v2"

dataset = load_dataset(dataset_path)

Generating train split:   0%|          | 0/130319 [-1:59:59<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [21]:
dataset["train"][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

## 预处理数据


In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)

以下断言确保我们的Tokenizers使用的是FastTokenizer

In [23]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

可以直接在两个句子上调用此比机器（一个用于答案，一个用于上下文）

In [24]:
tokenizer("What is your name?", "My name is Hengzq.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 21863, 2290, 2480, 4160, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## 训练超参数设置


In [29]:
from transformers import TrainingArguments

batch_size=64

training_args = TrainingArguments(
    output_dir =  f"{model_path}/test_trainer",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

## 实例化模型训练（Trainer）

In [30]:
from transformers import Trainer,AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_path)

trainer = Trainer(
    model,
    training_args,
    train_dataset = dataset["train"],
    eval_dataset = dataset["validation"]
)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at /home/hengzq/workspace/modelscope/models/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
trainer.train()

RuntimeError: Could not infer dtype of dict